<a href="https://colab.research.google.com/github/gabrielefrn/qc-minicourse/blob/main/Grover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

09/04/2024